In [1]:
import os
import numpy as np
import mido
NUM_TRACKS = 3
NUM_TIMESHIFTS = 4
NUM_MIDI_PITCHES = 128

# Get note sequences for each track.

# Get final sequences of vectors.

In [2]:
def get_tempo(midi):
    """Get tempo from MidiFile object.

    Args:
       midi: MidiFile object.
       
    Returns:
        Tempo in microseconds.
    """
    tempo = -1
    for i, track in enumerate(midi.tracks):
        for msg in track:
            if msg.is_meta and msg.type == 'set_tempo':
                msg_str = str(msg)
                start = msg_str.index('tempo=')
                new_msg = msg_str[start + 6:]
                end = new_msg.index(' ')
                tempo = int(new_msg[:end])

    return tempo

def get_note_sequences(midi_file):
    """Get note sequences for each track from a MidiFile object.

    Args:
       midi_file: Filename.
       
    Returns:
        List of tracks containing note sequences, where each note is a dictionary containing the note type
        ('note_on' or 'note_off'), the MIDI pitch value, and duration of the note in beats.
    """
    midi = mido.MidiFile(midi_file)
    tracks = []

    for track in midi.tracks:
        note_sequences = []
        for msg in track:               
            if not msg.is_meta:
                note_vector = {}
                num_beats = msg.time / midi.ticks_per_beat 
                if msg.type == 'note_on' or msg.type == 'note_off':
                    # Get note value.
                    msg_str = str(msg)
                    start = msg_str.index('note=')
                    new_msg = msg_str[start + 5:]
                    end = new_msg.index(' ')
                    note = int(new_msg[:end])
                    #if (note == 79):
                    #    print(msg.time, midi.ticks_per_beat, num_beats)
                    
                    # Get velocity.
                    start = msg_str.index('velocity=')
                    new_msg = msg_str[start + 9:]
                    end = new_msg.index(' ')
                    velocity = int(new_msg[:end])

                    # Set note vector values.
                    note_vector["type"] = msg.type
                    if velocity == 0:
                        note_vector["type"] = 'note_off'
                    note_vector["note"] = note
                    note_vector["time"] = num_beats

                    # Add note vector to note sequence list.
                    note_sequences.append(note_vector)
        tracks.append(note_sequences)

    return tracks

In [29]:
def beat_number_to_binary(n):
    return [(n // 8) % 2, (n // 4) % 2, (n // 2) % 2, n % 2]

def tracks_to_vector_sequence(tracks):
    """Get final vector from note sequence dictionaries.

    Args:
       tracks: List of tracks containing note sequence dictionaries with type (note_on or note_off),
       note, and time delay before note. Tracks are sorted from highest to lowest.
       
    Returns:
        Final one-hot vector containing NUM_MIDI_PITCHES * NUM_TRACKS note_on events,
        NUM_MIDI_PITCHES * NUM_TRACKS note_off events,
        and NUM_TIMESHIFTS timeshift events in intervals of 1/4 of a beat each. Also returns sequence of
        beat number vectors, represented as a length-4 vector in binary for each number from 0 to 15.
    """
    if tracks == None:
        return []
    
    assert len(tracks) <= NUM_TRACKS + 1
    
    # Sort all events by start time.
    events = []
    for i, sequence in enumerate(tracks[1:]):
        start_time = 0
        for event in sequence:
            start_time += event["time"]
            # Round to nearest sixteenth note.
            rounded_start_time = np.round(start_time * NUM_TIMESHIFTS) / NUM_TIMESHIFTS
            new_event = {"type": event["type"], "track": i, "note": event["note"],
                         "start_time": rounded_start_time}
            events.append(new_event)
    events = np.random.permutation(events)
    events = sorted(events, key=lambda x: x["start_time"])
    
    # Fix time lengths.
    prev_start_time = 0
    for e in events:
        e["time"] = e["start_time"] - prev_start_time
        prev_start_time = e["start_time"]
    
    # Create final vector sequence and beat number sequence.
    final_sequence = []
    beat_numbers = []
    current_beat = 0
    for e in events:
        #time_vector = np.zeros(NUM_MIDI_PITCHES * 2 * NUM_TRACKS + NUM_TIMESHIFTS)
        time = e["time"]
        # Create rest vectors if time is greater than 1 beat.
        while time > 1.0:
            #rest_vector = np.zeros(NUM_MIDI_PITCHES * 2 * NUM_TRACKS + NUM_TIMESHIFTS)
            #rest_vector[-1] = 1
            #final_sequence.append(rest_vector)
            final_sequence.append(NUM_MIDI_PITCHES * 2 * NUM_TRACKS + NUM_TIMESHIFTS - 1)
            beat_numbers.append(current_beat)
            current_beat = (current_beat + 4) % 16
            time -= 1.0
        if time > 0:
            timeshift = int(time * NUM_TIMESHIFTS) - 1
            #time_vector[NUM_MIDI_PITCHES * 2 * NUM_TRACKS + timeshift] = 1
            #final_sequence.append(time_vector)
            final_sequence.append(NUM_MIDI_PITCHES * 2 * NUM_TRACKS + timeshift)
            beat_numbers.append(current_beat)
        
        # Update current beat.
        current_beat = int(current_beat + time * 4) % 16    
        
        note_vector = np.zeros(NUM_MIDI_PITCHES * 2 * NUM_TRACKS + NUM_TIMESHIFTS)
        track_offset = NUM_MIDI_PITCHES * e["track"]
        # Set correct note.
        if e["type"] == 'note_on':
            #note_vector[track_offset + e["note"]] = 1
            final_sequence.append(track_offset + e["note"])
            beat_numbers.append(current_beat)
        else:
            #note_vector[NUM_MIDI_PITCHES * NUM_TRACKS + track_offset + e["note"]] = 1
            final_sequence.append(NUM_MIDI_PITCHES * NUM_TRACKS + track_offset + e["note"])
            beat_numbers.append(current_beat)
        #final_sequence.append(note_vector)
        
    return final_sequence, beat_numbers

# Use the functions below to get midi to vector sequence.

In [30]:
def midi_to_vector(filename):
    """Get final vector from midi file.

    Args:
       filename: File name of midi.
       
    Returns:
        Final one-hot vector containing NUM_MIDI_PITCHES * NUM_TRACKS note_on events,
        NUM_MIDI_PITCHES * NUM_TRACKS note_off events,
        and NUM_TIMESHIFTS timeshift events in intervals of 1/4 of a beat each.
    """
    tracks = get_note_sequences(filename)
    return tracks_to_vector_sequence(tracks)

In [39]:
def get_training_data(start_index, end_index, n_input=100, training_filename='training_data.txt'):
    """Generate training data array for a subset of files in "midis_processed_nopercussion/" directory.
    
    Args:
       start_index: Start index of training data.
       end_index: End index of training data.
       n_input: Minimum length of sequence of notes.
       training_filename: Name of output text file that contains training data.
    """
    directory = 'midis_processed_nopercussion/'
    training_data_file = open(training_filename, 'w')
    beat_number_file = open('beat_number_' + training_filename, 'w')
    for i, filename in enumerate(os.listdir(directory)[start_index:end_index]):
        try:
            if filename.endswith(".mid"):
                midi = mido.MidiFile(directory + filename)
                vector, beats = midi_to_vector(directory + filename)
                assert len(vector) == len(beats)
                if len(vector) > n_input:
                    training_output_string = filename + '~'
                    beat_output_string = filename + '~'
                    for v, b in zip(vector, beats):
                        if v >= 772:
                            print("ERROR: " + str(v) + " " + filename)
                        training_output_string += ' ' + str(v)
                        beat_output_string += ' ' + str(b)
                    training_data_file.write(training_output_string + '\n')
                    beat_number_file.write(beat_output_string + '\n')
                else:
                    print("Training data too short: " + filename + ". len(vector)=" + str(len(vector)))
        except:
            print("Faulty training data: " + filename + ". len(vector)=" + str(len(vector)))
        if (i+1)%25 == 0:
            print(" > {}/{} files loaded.".format(i+1, len(os.listdir(directory)[start_index:end_index])))
               
    training_data_file.close()
    beat_number_file.close()

In [43]:
def get_training_vectors_from_file(training_filename, beat_filename):
    """Generate vectors from training data text file.
    
    Args:
       training_filename: Name of text file that contains training data.
       beat_filename: Name of beat numbers text file.
       
    Returns:
        Numpy arrays of training data, training data labels, and beat numbers.
    """
    training_data_file = open(training_filename, 'r')
    beat_file = open(beat_filename, 'r')
    training_data = []
    training_data_labels = []
    beat_numbers = []
    for line in training_data_file:
        index = line.index('~')
        filename = line[:index]
        tokens = line[index + 2:].strip().split(' ')
        vector = np.array(list(map(int, tokens)))
        for v in vector:
            if v > 772:
                print("ERROR: " + str(v) + ' ' + filename)
        training_data.append(np.array(vector))
        training_data_labels.append(filename)
        
    for line in beat_file:
        index = line.index('~')
        filename = line[:index]
        tokens = line[index + 2:].strip().split(' ')
        vector = np.array(list(map(int, tokens)))
        for v in vector:
            if v >= 16 or v < 0:
                print("ERROR: " + str(v) + ' ' + filename)
        beat_numbers.append(np.array(vector))
        
    return np.array(training_data), training_data_labels, beat_numbers

In [42]:
get_training_data(0, 6000)

 > 25/5824 files loaded.
Training data too short: PkmRB-Item2.mid. len(vector)=50
 > 50/5824 files loaded.
 > 75/5824 files loaded.
 > 100/5824 files loaded.
Training data too short: trojintr.mid. len(vector)=87
 > 125/5824 files loaded.
Faulty training data: DG_SMB_Main.mid. len(vector)=1179
 > 150/5824 files loaded.
 > 175/5824 files loaded.
 > 200/5824 files loaded.
 > 225/5824 files loaded.
 > 250/5824 files loaded.
Training data too short: Gpunchoutboss.mid. len(vector)=72
 > 275/5824 files loaded.
Training data too short: Zelda_-_Game_Over.mid. len(vector)=96
 > 300/5824 files loaded.
Training data too short: immortatatk.mid. len(vector)=45
 > 325/5824 files loaded.
 > 350/5824 files loaded.
 > 375/5824 files loaded.
Training data too short: Flag.mid. len(vector)=81
 > 400/5824 files loaded.
Training data too short: metroidit.mid. len(vector)=59
 > 425/5824 files loaded.
 > 450/5824 files loaded.
Training data too short: rcproam_startrace.mid. len(vector)=97
 > 475/5824 files loa

 > 4575/5824 files loaded.
Training data too short: Mm6vict2.mid. len(vector)=94
 > 4600/5824 files loaded.
 > 4625/5824 files loaded.
 > 4650/5824 files loaded.
 > 4675/5824 files loaded.
 > 4700/5824 files loaded.
 > 4725/5824 files loaded.
Training data too short: cvLostLife.mid. len(vector)=64
 > 4750/5824 files loaded.
Training data too short: Metrma.mid. len(vector)=77
 > 4775/5824 files loaded.
 > 4800/5824 files loaded.
 > 4825/5824 files loaded.
 > 4850/5824 files loaded.
Training data too short: zelda-go.mid. len(vector)=70
Faulty training data: DG_MM5_Darkman.mid. len(vector)=374
 > 4875/5824 files loaded.
Training data too short: mm3protw.mid. len(vector)=80
 > 4900/5824 files loaded.
 > 4925/5824 files loaded.
Training data too short: POtbout.mid. len(vector)=72
 > 4950/5824 files loaded.
Training data too short: Deleting_Photo.mid. len(vector)=75
 > 4975/5824 files loaded.
 > 5000/5824 files loaded.
Training data too short: STANLevel_Complete.mid. len(vector)=75
Training 